In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_456584/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
spark = SparkSession.builder.master("yarn").getOrCreate()

In [20]:
data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40],
        [5, 11, 1000, 11, 11],
        [6, 11, 2000, 12, 12],
        ]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})

df_person = spark.createDataFrame(insurance)
df_person.createOrReplaceTempView("Insurance")

data_1 = [


[ 1   , 224.17   , 952.73   , 32.4 , 20.2 ],
[ 2   , 224.17   , 900.66   , 52.4 , 32.7 ],
[ 3   , 824.61   , 645.13   , 72.4 , 45.2 ],
[ 4   , 424.32   , 323.66   , 12.4 , 7.7  ],
[ 5   , 424.32   , 282.9    , 12.4 , 7.7  ],
[ 6   , 625.05   , 243.53   , 52.5 , 32.8 ],
[ 7   , 424.32   , 968.94   , 72.5 , 45.3 ],
[ 8   , 624.46   , 714.13   , 12.5 , 7.8  ],
[ 9   , 425.49   , 463.85   , 32.5 , 20.3 ],
[ 10  , 624.46   , 776.85   , 12.4 , 7.7  ],
[ 11  , 624.46   , 692.71   , 72.5 , 45.3 ],
[ 12  , 225.93   , 933      , 12.5 , 7.8  ],
[ 13  , 824.61   , 786.86   , 32.6 , 20.3 ],
[ 14  , 824.61   , 935.34   , 52.6 , 32.8 ],
    
    
]

insurance = pd.DataFrame(data_1, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})

df_person = spark.createDataFrame(insurance)
df_person.createOrReplaceTempView("Insurance_1")

In [21]:
insurance

,pid,tiv_2015,tiv_2016,lat,lon
0,1,224.17,952.73,32.4,20.2
1,2,224.17,900.66,52.4,32.7
2,3,824.61,645.13,72.4,45.2
3,4,424.32,323.66,12.4,7.7
4,5,424.32,282.9,12.4,7.7
5,6,625.05,243.53,52.5,32.8
6,7,424.32,968.94,72.5,45.3
7,8,624.46,714.13,12.5,7.8
8,9,425.49,463.85,32.5,20.3
9,10,624.46,776.85,12.4,7.7


In [28]:
query = """
    with unique_lat_lon as (
        select
            *
        from (
            select
                *,
                count(*) over (partition by lat, lon) as count_same_lat_lon,
                count(*) over (partition by tiv_2015) as count_same_2015
            from Insurance_1 i
        ) where count_same_lat_lon = 1 and count_same_2015 >= 2
    )
    
    select sum(tiv_2016) as tiv_2016 from unique_lat_lon

"""

In [26]:
query = """
    with unique_lat_lon as (
        select
            *
        from (
            select
                *,
                count(*) over (partition by lat, lon) as count_same_lat_lon,
                count(*) over (partition by tiv_2015) as count_same_2015
            from Insurance_1 i
        ) 
    )
    
    select *  from unique_lat_lon order by pid asc

"""

In [29]:
spark.sql(query).show()

+--------+
|tiv_2016|
+--------+
| 4220.72|
+--------+

